## Accumulate and Decay <sup>10</sup>Be
<br>
  
#### Syntax
`conc10Be = Be10AnnualFxn(conc0, CosmoPars)` <br>
#### Input 
`conc0` : vector of size len(CRNz) with spacing dz containing the initial concentration (at g-1).
`CosmoPars` : dictionary of cosmogenic nuclide accumulation/decay parameters. <br>

#### Variables Used
##### Cosmo Pars
`CRNz` : depth vector of len(max_depths / dz) <br>
`SLHLBe10` : SLHL 10Be production rate (at g-1 yr-1) <br>
`scaling_factor` : CN scaling factor (e.g. Stone 2000) <br> 
`Be10_decay` : Be10 decay constant (yr-1)<br>
`B_Be` : pre-exponential term for Be muogenic production <br>
`L3` : fast muon attenuation length Be10 (cm-2) <br>
`rock_rho` :  = bulk rock density (g cm-3) <br>
`mu` : att length / density <br>

#### Output
`conc10Be` : vector of size len(CRNz) with spacing dz containing concentrations after this additional year (at g-1).<br>
#### Notes
**Date of Creation:** 1. Juli 2021 <br>
**Author:** Donovan Dennis <br>
**Update:** <br>
04.03.2022 : Renamed from Be10AnnualFxnmtest to Be10AnnualFxn for publication. <br>

In [4]:
def Be10AnnualFxn(conc0, muon0, spall0, CosmoPars):
    # import the relevant cosmo acc/decay parameters
    CRNz = CosmoPars['CRNz']
    SLHLBe10 = CosmoPars['SLHL_Be10']
    scaling_factor = CosmoPars['scaling_factor']
    Be10_decay = CosmoPars['Be10 Decay']
    B_Be = CosmoPars['B_Be']
    L3 = CosmoPars['L3 Be']
    rock_rho = CosmoPars['rock_rho']
    mu = CosmoPars['mu']
    
    # calculate site-specific production rate atoms-1 g-1  yr-1
    Be10_P0 = SLHLBe10 * scaling_factor
    
    # loop to determine fast muon concentration at depth CRNz[i]
    fast_muons_10 = [B_Be * np.e**(-rock_rho * CRNz[i] / L3) for i in range(len(CRNz))]
    # and loop again for spallogenic concentration at depth CRNz[i]
    spallogenic_10 = [Be10_P0 * np.e**(-mu * CRNz[i]) for i in range(len(CRNz))]
    # grids match so can directly add concentrations to each other
    production_tot_10 = [fast_muons_10[i] + spallogenic_10[i] for i in range(len(CRNz))]
    
    # return the total concentration after Be-10 decay
    muon10Be = [muon0[i] * np.e**(-Be10_decay) + (fast_muons_10[i] / (Be10_decay + 0) * (1 - np.e**(-(Be10_decay + 0) * 1))) for i in range(len(CRNz))] 
    spall10Be = [spall0[i] * np.e**(-Be10_decay) + (spallogenic_10[i] / (Be10_decay + 0) * (1 - np.e**(-(Be10_decay + 0) * 1))) for i in range(len(CRNz))] 
    
    conc10Be = [conc0[i] * np.e**(-Be10_decay) + (production_tot_10[i] / (Be10_decay + 0) * (1 - np.e**(-(Be10_decay + 0) * 1))) for i in range(len(CRNz))]    

    return conc10Be, muon10Be, spall10Be, 